# Applied Machine Learning homework3

Huibo Zhao hz2480
Lingjie Xu lx2222

# Perform General Data Preprocessing

In [54]:
import os
import pandas as pd

In [55]:
# Extract those files that contain our training data
path = os.getcwd()
files = os.listdir(path)
xlsx_files = [f for f in files if f[-4:] == 'xlsx' and f[0] != '~']
xlsx_files

['2015 FE Guide-for DOE-Mobility Ventures only-OK to release-no-sales-4-27-2017Mercedesforpublic.xlsx',
 '2016 FE Guide for DOE-OK to release-no-sales-4-27-2017Mercedesforpublic.xlsx',
 '2017 FE Guide for DOE-release dates before 9-20-2017-no sales-9-19-2017MercedesCadillacforpublic.xlsx']

In [56]:
# Putting all the contents into dataframe for viewing and accessing
df = pd.DataFrame()
for f in xlsx_files:
    data = pd.read_excel(f, 'FEguide')
    df = df.append(data)

In [57]:
df

,Model Year,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,City FE (Guide) - Conventional Fuel,...,City CO2 Rounded Adjusted,Hwy CO2 Rounded Adjusted,Comb CO2 Rounded Adjusted (as shown on FE Label),CO2-PHEV Composite Rounded Adjusted Combined CO2,240V Charge Time at 240 volts (hours),120V Charge time at 120 Volts (hours),PHEV Total Driving Range (rounded to nearest 10 miles)DISTANCE,City PHEV Composite MPGe,Hwy PHEV Composite MPGe,Comb PHEV Composite MPGe
0,2015,FCA Italy,Alfa Romeo,4C,FTG,264,1.8,4,Auto(AM6),24,...,365,259,317,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,aston martin,Aston Martin Lagonda Ltd,V12 Vantage S,ASX,8,6.0,12,Auto(AM7),12,...,742,488,628,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,ASX,4,4.7,8,Auto(AM7),14,...,655,426,552,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,ASX,1,4.7,8,Manual(M6),13,...,681,461,582,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,ASX,5,4.7,8,Auto(AM7),14,...,655,426,552,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,ASX,2,4.7,8,Manual(M6),13,...,681,461,582,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015,aston martin,Aston Martin Lagonda Ltd,Vantage GT,ASX,6,4.7,8,Auto(AM7),14,...,655,426,552,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2015,aston martin,Aston Martin Lagonda Ltd,Vantage GT,ASX,3,4.7,8,Manual(M6),13,...,681,461,582,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2015,Volkswagen Group of,Audi,R8,VGA,27,4.2,8,Auto(AM-S7),14,...,656,394,538,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2015,Volkswagen Group of,Audi,R8,VGA,29,4.2,8,Manual(M6),11,...,772,439,622,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
df.columns

Index(['Model Year', 'Mfr Name', 'Division', 'Carline', 'Verify Mfr Cd',
       'Index (Model Type Index)', 'Eng Displ', '# Cyl', 'Transmission',
       'City FE (Guide) - Conventional Fuel',
       ...
       'City CO2 Rounded Adjusted', 'Hwy CO2 Rounded Adjusted',
       'Comb CO2 Rounded Adjusted (as shown on FE Label)',
       'CO2-PHEV Composite Rounded Adjusted Combined CO2',
       '240V Charge Time at 240 volts (hours)',
       '120V Charge time at 120 Volts (hours)',
       'PHEV Total Driving Range (rounded to nearest 10 miles)DISTANCE',
       'City PHEV Composite MPGe', 'Hwy PHEV Composite MPGe',
       'Comb PHEV Composite MPGe'],
      dtype='object', length=162)

In [59]:
import numpy as np
da = np.array(df.columns)

In [60]:
np.where(da == 'Comb Unrd Adj FE - Conventional Fuel')

(array([17]),)

In [61]:
da

array(['Model Year', 'Mfr Name', 'Division', 'Carline', 'Verify Mfr Cd',
       'Index (Model Type Index)', 'Eng Displ', '# Cyl', 'Transmission',
       'City FE (Guide) - Conventional Fuel',
       'Hwy FE (Guide) - Conventional Fuel',
       'Comb FE (Guide) - Conventional Fuel',
       'City Unadj FE - Conventional Fuel',
       'Hwy Unadj FE - Conventional Fuel',
       'Comb Unadj FE - Conventional Fuel',
       'City Unrd Adj FE - Conventional Fuel',
       'Hwy Unrd Adj FE - Conventional Fuel',
       'Comb Unrd Adj FE - Conventional Fuel', 'Guzzler? ',
       'Air Aspir Method', 'Air Aspiration Method Desc', 'Trans',
       'Trans Desc', 'Trans, Other', '# Gears', 'Lockup Torque Converter',
       'Trans Creeper Gear', 'Drive Sys', 'Drive Desc',
       'Max Ethanol % - Gasoline', 'Max Biodiesel %',
       'Range1 - Model Type Driving Range - Conventional Fuel',
       'Fuel Usage  - Conventional Fuel',
       'Fuel Usage Desc - Conventional Fuel',
       'Fuel Unit - Convention

In [62]:
# Professor had named a few features that we cannot use, we extract the name of those features
drop_index = list()
for i in range(len(da)):
    if "FE" in da[i]:
        drop_index.append(da[i])
    if "MPG" in da[i]:
        drop_index.append(da[i])
    if "CO2" in da[i]:
        drop_index.append(da[i])
    if "Smog" in da[i]:
        drop_index.append(da[i])
    if "Guzzler" in da[i]:
        drop_index.append(da[i])
    if "EPA" in da[i]:
        drop_index.append(da[i])
len(drop_index)

66

In [63]:
# Drop those disallowed features
df_ = df.drop(drop_index, axis=1)
df_

,Model Year,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,Air Aspir Method,...,Charge Depleting Calc Appr Desc (PHEV only),Charge Sustaining Calc Appr Code (PHEV only),Charge Sustaining Calc Appr Desc (PHEV only),Unnamed: 130,GHG Rating (1-10 rating on Label),$ You Save over 5 years (amount saved in fuel costs over 5 years - on label),$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label),240V Charge Time at 240 volts (hours),120V Charge time at 120 Volts (hours),PHEV Total Driving Range (rounded to nearest 10 miles)DISTANCE
0,2015,FCA Italy,Alfa Romeo,4C,FTG,264,1.8,4,Auto(AM6),TC,...,NaN,NaN,NaN,NaN,7,750.0,NaN,NaN,NaN,NaN
1,2015,aston martin,Aston Martin Lagonda Ltd,V12 Vantage S,ASX,8,6.0,12,Auto(AM7),NaN,...,NaN,NaN,NaN,NaN,2,NaN,9250.0,NaN,NaN,NaN
2,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,ASX,4,4.7,8,Auto(AM7),NaN,...,NaN,NaN,NaN,NaN,3,NaN,6750.0,NaN,NaN,NaN
3,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,ASX,1,4.7,8,Manual(M6),NaN,...,NaN,NaN,NaN,NaN,3,NaN,8000.0,NaN,NaN,NaN
4,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,ASX,5,4.7,8,Auto(AM7),NaN,...,NaN,NaN,NaN,NaN,3,NaN,6750.0,NaN,NaN,NaN
5,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,ASX,2,4.7,8,Manual(M6),NaN,...,NaN,NaN,NaN,NaN,3,NaN,8000.0,NaN,NaN,NaN
6,2015,aston martin,Aston Martin Lagonda Ltd,Vantage GT,ASX,6,4.7,8,Auto(AM7),NaN,...,NaN,NaN,NaN,NaN,3,NaN,6750.0,NaN,NaN,NaN
7,2015,aston martin,Aston Martin Lagonda Ltd,Vantage GT,ASX,3,4.7,8,Manual(M6),NaN,...,NaN,NaN,NaN,NaN,3,NaN,8000.0,NaN,NaN,NaN
8,2015,Volkswagen Group of,Audi,R8,VGA,27,4.2,8,Auto(AM-S7),NaN,...,NaN,NaN,NaN,NaN,4,NaN,5750.0,NaN,NaN,NaN
9,2015,Volkswagen Group of,Audi,R8,VGA,29,4.2,8,Manual(M6),NaN,...,NaN,NaN,NaN,NaN,2,NaN,9250.0,NaN,NaN,NaN


In [52]:
da_ = np.array(df_.columns)
da_

array(['Model Year', 'Mfr Name', 'Division', 'Carline', 'Verify Mfr Cd',
       'Index (Model Type Index)', 'Eng Displ', '# Cyl', 'Transmission',
       'Air Aspir Method', 'Air Aspiration Method Desc', 'Trans',
       'Trans Desc', 'Trans, Other', '# Gears', 'Lockup Torque Converter',
       'Trans Creeper Gear', 'Drive Sys', 'Drive Desc',
       'Max Ethanol % - Gasoline', 'Max Biodiesel %',
       'Range1 - Model Type Driving Range - Conventional Fuel',
       'Fuel Usage  - Conventional Fuel',
       'Fuel Usage Desc - Conventional Fuel',
       'Fuel Unit - Conventional Fuel',
       'Fuel Unit Desc - Conventional Fuel', '2Dr Pass Vol',
       '2Dr Lugg Vol', '4Dr Pass Vol', '4Dr Lugg Vol', 'Htchbk Pass Vol',
       'Htchbk Lugg Vol', 'Annual Fuel1 Cost - Conventional Fuel',
       ' Range2 - Alt Fuel Model Typ Driving Range - Alternative Fuel',
       ' Fuel2 Usage - Alternative Fuel',
       ' Fuel2 Usage Desc - Alternative Fuel',
       'Fuel2 Unit - Alternative Fuel',
       '

In [64]:
df.shape

(3701, 162)

In [65]:
X = df_.as_matrix()
X.shape

(3701, 106)

In [82]:
# We extract the target value
y = df['Comb Unrd Adj FE - Conventional Fuel'].as_matrix()


# Examine if the target contains any empty or nan value
# If it does, we need to drop the row without a valid target value
# Fortunately, we don't find such row
for i in y:
    if isinstance(i,float) and math.isnan(i):
        print(i)

In [68]:
# We examine each feature and count how many nan values it have
# Save results in a dictionary
import math
type(df_.columns.values)
column_names = df_.columns.values
type(column_names)

nan_dict = {}


for column_name in column_names:
    print(column_name)
    count = 0
    #print(df.iloc[0][column_name])

    for i in range(0,df_.shape[0]):
        if isinstance(df_.iloc[i][column_name],float) and math.isnan(df_.iloc[i][column_name]):
            count += 1
    nan_dict[column_name] = count


    
print(nan_dict)

Model Year
Mfr Name
Division
Carline
Verify Mfr Cd
Index (Model Type Index)
Eng Displ
# Cyl
Transmission
Air Aspir Method
Air Aspiration Method Desc
Trans
Trans Desc
Trans, Other
# Gears
Lockup Torque Converter
Trans Creeper Gear
Drive Sys
Drive Desc
Max Ethanol % - Gasoline
Max Biodiesel %
Range1 - Model Type Driving Range - Conventional Fuel
Fuel Usage  - Conventional Fuel
Fuel Usage Desc - Conventional Fuel
Fuel Unit - Conventional Fuel
Fuel Unit Desc - Conventional Fuel
2Dr Pass Vol
2Dr Lugg Vol
4Dr Pass Vol
4Dr Lugg Vol
Htchbk Pass Vol
Htchbk Lugg Vol
Annual Fuel1 Cost - Conventional Fuel
 Range2 - Alt Fuel Model Typ Driving Range - Alternative Fuel
 Fuel2 Usage - Alternative Fuel
 Fuel2 Usage Desc - Alternative Fuel
Fuel2 Unit - Alternative Fuel
Fuel2 Unit Desc - Alternative Fuel
Fuel2 Annual Fuel Cost - Alternative Fuel
Descriptor - Model Type (40 Char or less)
Intake Valves Per Cyl
Exhaust Valves Per Cyl
Carline Class
Carline Class Desc
Car/Truck Category - Cash for Clunkers Bi

In [74]:
# we plan to drop those features who have more
# than 2000 nonsense values (3701 rows in total)

drop_ = list()
for key in nan_dict:
    if nan_dict[key]>=2000:
        drop_.append(key)
        
len(drop_)

58

In [75]:
df_1 = df_.drop(drop_, axis=1)
df_1

,Model Year,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,Air Aspir Method,...,Var Valve Timing Desc,Var Valve Lift?,Fuel Metering Sys Cd,Fuel Metering Sys Desc,Camless Valvetrain (Y or N),Oil Viscosity,Stop/Start System (Engine Management System) Code,Stop/Start System (Engine Management System) Description,GHG Rating (1-10 rating on Label),$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label)
0,2015,FCA Italy,Alfa Romeo,4C,FTG,264,1.8,4,Auto(AM6),TC,...,Cam Phasers,N,GDI,Spark Ignition Direct Injection,N,5W-30,N,No,7,NaN
1,2015,aston martin,Aston Martin Lagonda Ltd,V12 Vantage S,ASX,8,6.0,12,Auto(AM7),NaN,...,Inlet and Exhaust Cam Phasing,N,MFI,Multipoint/sequential fuel injection,N,0W40,N,No,2,9250.0
2,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,ASX,4,4.7,8,Auto(AM7),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,6750.0
3,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,ASX,1,4.7,8,Manual(M6),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,8000.0
4,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,ASX,5,4.7,8,Auto(AM7),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,6750.0
5,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,ASX,2,4.7,8,Manual(M6),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,8000.0
6,2015,aston martin,Aston Martin Lagonda Ltd,Vantage GT,ASX,6,4.7,8,Auto(AM7),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,6750.0
7,2015,aston martin,Aston Martin Lagonda Ltd,Vantage GT,ASX,3,4.7,8,Manual(M6),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,8000.0
8,2015,Volkswagen Group of,Audi,R8,VGA,27,4.2,8,Auto(AM-S7),NaN,...,CONTINUOUS INTAKE AND EXHAUST CAM ADJUSTEMENT,N,GDI,Spark Ignition Direct Injection,N,"5W/30, VW50400",N,No,4,5750.0
9,2015,Volkswagen Group of,Audi,R8,VGA,29,4.2,8,Manual(M6),NaN,...,CONTINUOUS INTAKE AND EXHAUST CAM ADJUSTEMENT,N,GDI,Spark Ignition Direct Injection,N,"5W/30, VW50400",N,No,2,9250.0


In [76]:
df_1.to_csv('11.csv')

In [84]:
type(df_1.columns.values)
column_names = df_1.columns.values
type(column_names)

count_dict = {}


for column_name in column_names:
    print(column_name)
    count = 0
    #print(df.iloc[0][column_name])

    for i in range(0,df_1.shape[0]):
        if isinstance(df_1.iloc[i][column_name],float) and math.isnan(df_1.iloc[i][column_name]):
            count += 1
    count_dict[column_name] = count


    
#print(nan_dict)


test = list()
for key in count_dict:
    if count_dict[key]> 0:
        test.append(key)
        
print(test)

Model Year
Mfr Name
Division
Carline
Verify Mfr Cd
Index (Model Type Index)
Eng Displ
# Cyl
Transmission
Air Aspir Method
Air Aspiration Method Desc
Trans
Trans Desc
# Gears
Lockup Torque Converter
Trans Creeper Gear
Drive Sys
Drive Desc
Max Ethanol % - Gasoline
Fuel Usage  - Conventional Fuel
Fuel Usage Desc - Conventional Fuel
Fuel Unit - Conventional Fuel
Fuel Unit Desc - Conventional Fuel
Annual Fuel1 Cost - Conventional Fuel
Descriptor - Model Type (40 Char or less)
Intake Valves Per Cyl
Exhaust Valves Per Cyl
Carline Class
Carline Class Desc
Car/Truck Category - Cash for Clunkers Bill.
Calc Approach Desc
Release Date
Unique Label?
Label Recalc?
Suppressed?
Police/Emerg?
Cyl Deact?
Var Valve Timing?
Var Valve Timing Desc
Var Valve Lift?
Fuel Metering Sys Cd
Fuel Metering Sys Desc
Camless Valvetrain (Y or N)
Oil Viscosity
Stop/Start System (Engine Management System) Code
Stop/Start System (Engine Management System)  Description
GHG Rating (1-10 rating on Label)
$ You Spend over 5 y

In [90]:
for key in count_dict:
    if count_dict[key]> 0:
        print(str(key) + ": " + str(count_dict[key]))

Air Aspir Method: 1973
Max Ethanol % - Gasoline: 80
Descriptor - Model Type (40 Char or less): 1131
Car/Truck Category - Cash for Clunkers Bill.: 1242
Unique Label?: 295
Label Recalc?: 187
Var Valve Timing Desc: 107
$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label) : 1080


In [78]:
for i in test:
    print(i)

Air Aspir Method
Max Ethanol % - Gasoline
Descriptor - Model Type (40 Char or less)
Car/Truck Category - Cash for Clunkers Bill.
Unique Label?
Label Recalc?
Var Valve Timing Desc
$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label) 


In [92]:
df_2 = df_1.drop("Car/Truck Category - Cash for Clunkers Bill.", axis=1)
df_2

,Model Year,Mfr Name,Division,Carline,Verify Mfr Cd,Index (Model Type Index),Eng Displ,# Cyl,Transmission,Air Aspir Method,...,Var Valve Timing Desc,Var Valve Lift?,Fuel Metering Sys Cd,Fuel Metering Sys Desc,Camless Valvetrain (Y or N),Oil Viscosity,Stop/Start System (Engine Management System) Code,Stop/Start System (Engine Management System) Description,GHG Rating (1-10 rating on Label),$ You Spend over 5 years (increased amount spent in fuel costs over 5 years - on label)
0,2015,FCA Italy,Alfa Romeo,4C,FTG,264,1.8,4,Auto(AM6),TC,...,Cam Phasers,N,GDI,Spark Ignition Direct Injection,N,5W-30,N,No,7,NaN
1,2015,aston martin,Aston Martin Lagonda Ltd,V12 Vantage S,ASX,8,6.0,12,Auto(AM7),NaN,...,Inlet and Exhaust Cam Phasing,N,MFI,Multipoint/sequential fuel injection,N,0W40,N,No,2,9250.0
2,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,ASX,4,4.7,8,Auto(AM7),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,6750.0
3,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage,ASX,1,4.7,8,Manual(M6),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,8000.0
4,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,ASX,5,4.7,8,Auto(AM7),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,6750.0
5,2015,aston martin,Aston Martin Lagonda Ltd,V8 Vantage S,ASX,2,4.7,8,Manual(M6),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,8000.0
6,2015,aston martin,Aston Martin Lagonda Ltd,Vantage GT,ASX,6,4.7,8,Auto(AM7),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,6750.0
7,2015,aston martin,Aston Martin Lagonda Ltd,Vantage GT,ASX,3,4.7,8,Manual(M6),NaN,...,"Electronically controlled, hydraulically actua...",N,MFI,Multipoint/sequential fuel injection,N,10W60,N,No,3,8000.0
8,2015,Volkswagen Group of,Audi,R8,VGA,27,4.2,8,Auto(AM-S7),NaN,...,CONTINUOUS INTAKE AND EXHAUST CAM ADJUSTEMENT,N,GDI,Spark Ignition Direct Injection,N,"5W/30, VW50400",N,No,4,5750.0
9,2015,Volkswagen Group of,Audi,R8,VGA,29,4.2,8,Manual(M6),NaN,...,CONTINUOUS INTAKE AND EXHAUST CAM ADJUSTEMENT,N,GDI,Spark Ignition Direct Injection,N,"5W/30, VW50400",N,No,2,9250.0
